#### SMOLLM-135M -  Base model

In [1]:
checkpoint = "HuggingFaceTB/SmolLM-135M"


In [2]:
from transformers import pipeline 

smol_pipe = pipeline("text-generation", model=checkpoint, device_map="auto")


Device set to use cuda:0


In [3]:
prompt_one = "Who are you?"
prompt_two = [{"role": "user", "content": prompt_one}]

smol_pipe(prompt_two)

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM

smol_tokenizer = AutoTokenizer.from_pretrained(checkpoint)
smol_model = AutoModelForCausalLM.from_pretrained(checkpoint)

In [7]:
### Load the dataset
from datasets import load_dataset

train_dataset = load_dataset("trl-lib/Capybara", split="train")
test_dataset = load_dataset("trl-lib/Capybara", split="test")

train_dataset, test_dataset

(Dataset({
     features: ['source', 'messages', 'num_turns'],
     num_rows: 15806
 }),
 Dataset({
     features: ['source', 'messages', 'num_turns'],
     num_rows: 200
 }))

In [11]:
from trl import SFTTrainer, SFTConfig

config = SFTConfig(
  chat_template_path="HuggingFaceTB/SmolLM3-3B",
  output_dir = "SmolLM2-Instruct",
  logging_steps=200,
  eval_steps =200,
  eval_strategy="steps"
)

trainer = SFTTrainer(model=smol_model, train_dataset=train_dataset, args=config, eval_dataset=test_dataset)


In [12]:
trainer.train()

Step,Training Loss,Validation Loss
200,1.194800,1.231385
400,1.206200,1.218086
600,1.198000,1.207791
800,1.200100,1.202613
1000,1.184200,1.198935
1200,1.183700,1.195314
1400,1.181200,1.193271
1600,1.185300,1.190670
1800,1.188500,1.188710
2000,1.163200,1.187245


TrainOutput(global_step=5928, training_loss=1.1485422471596, metrics={'train_runtime': 1267.4608, 'train_samples_per_second': 37.412, 'train_steps_per_second': 4.677, 'total_flos': 3.093890702628557e+16, 'train_loss': 1.1485422471596})

In [13]:
model_path = "./SmolLM2-Instruct/checkpoint-5928"
test_pipe =  pipeline("text-generation", model=model_path, device_map="auto")


Device set to use cuda:0


In [20]:
test_pipe(prompt_two)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant', 'content': 'I am a human.'}]}]